In [1]:
import numpy as np
import pandas as pd

import pystan
from sklearn.model_selection import train_test_split
import pickle
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from argparse import ArgumentParser, ArgumentTypeError
from pathlib import Path

In [2]:
class Adult_dataset():
    
    def __init__(self, train_file='adult_dataset/adult.data', 
                       test_file='adult_dataset/adult.test'):
        self.cols = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                     'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
                     'hours-per-week', 'native-country', 'salary']
        self.train_df = pd.read_csv(train_file, names=self.cols)
        self.test_df = pd.read_csv(test_file, names=self.cols)
        
        # Preprocessing 
        self.train_df = self.preprocessing(self.train_df)
        self.test_df = self.preprocessing(self.test_df)
        
    
    def preprocessing(self, df):
        # Cols to drop
        df = df.drop(columns=['fnlwgt', 'education'])
        
        # Remove rows with unknown data
        df = df.replace(' ?', np.NaN)
        df = df.dropna()
        
        # Convert categorical to int 
        df['salary'] = df['salary'].replace(' <=50K.', 0).replace(' >50K.', 1)
        df['salary'] = df['salary'].replace(' <=50K', 0).replace(' >50K', 1)
        df['workclass'] = df['workclass'].apply(lambda x: 1 if x==' Private' else 0)
        df['marital-status'] = df['marital-status'].apply(lambda x: 1 if x==' Married-civ-spouse' else 0) 
        df['relationship'] = df['relationship'].apply(lambda x: 1 if x in (' Husband', ' Wife') else 0) 
        occupation_mapping = {
            " Exec-managerial": 1,
            " Craft-repair": 0,
            " Prof-specialty": 1,
            " Sales": 1,
            " Adm-clerical": 1,
            " Other-service": 0.5,
            " Machine-op-inspct": 0,
            " Transport-moving": 0,
            " Handlers-cleaners": 0,
            " Tech-support": 1,
            " Farming-fishing": 0,
            " Protective-serv": 0,
            " Priv-house-serv": 0,
            " Armed-Forces": 0
        }
        df['occupation'] = df['occupation'].apply(lambda x: occupation_mapping[x]) 
        df['race'] = df['race'].apply(lambda x: 1 if x==' White' else 0)
        df['sex'] = df['sex'].apply(lambda x: 1 if x==' Male' else 0)
        df['native-country'] = df['native-country'].apply(lambda x: 1 if x==' United-States' else 0)
        
        # Normalize between 0 and 1
        df['age'] = df['age'] / df['age'].max()
        df['education-num'] = df['education-num'] / df['education-num'].max()
        df['capital-gain'] = df['capital-gain'] / df['capital-gain'].max()
        df['capital-loss'] = df['capital-loss'] / df['capital-loss'].max()
        df['hours-per-week'] = df['hours-per-week'] / df['hours-per-week'].max()
        
        return df

adult_ds = Adult_dataset()
adult_ds.test_df
adult_ds.test_df.describe()

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
count,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000,15060.000000
mean,0.430759,0.731806,0.632047,0.464143,0.591700,0.457371,0.861222,0.673772,0.011203,0.023619,0.413652,0.915538,0.245684
std,0.148674,0.443034,0.159920,0.498729,0.463801,0.498196,0.345726,0.468848,0.077033,0.107767,0.121847,0.278089,0.430506
min,0.188889,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010101,0.000000,0.000000
25%,0.311111,0.000000,0.562500,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.404040,1.000000,0.000000
50%,0.411111,1.000000,0.625000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.404040,1.000000,0.000000
75%,0.533333,1.000000,0.812500,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.454545,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Models

In [3]:
# wrapper class for statsmodels linear regression (more stable than SKLearn)
class SM_LinearRegression():
    def __init__(self):
        pass

    def fit(self, X, y):
        N = X.shape[0]
        self.LRFit = sm.OLS(y, np.hstack([X, np.ones(N).reshape(-1, 1)]), hasconst=True).fit()

    def predict(self, X):
        N = X.shape[0]
        return self.LRFit.predict(np.hstack([X, np.ones(N).reshape(-1, 1)]))

In [4]:
# Get the Unfair Model predictions
def Unfair_Model_Replication(train_df, test_df):
    lr_unfair = SM_LinearRegression()
    train_df_x = train_df.drop(columns=['salary']).to_numpy()
    train_df_y = train_df['salary'].to_numpy()
    lr_unfair.fit(train_df_x, train_df_y)
    
    test_df_x = test_df.drop(columns=['salary']).to_numpy()
    test_df_y = test_df['salary'].to_numpy()
    preds = lr_unfair.predict(test_df_x)

    # Return Results:
    return preds

# Get the Unaware Model predictions
def Unaware_Model_Replication(train_df, test_df, protected=['sex', 'race']):
    lr_unfair = SM_LinearRegression()
    train_df_x = train_df.drop(columns=['salary'] + protected).to_numpy()
    train_df_y = train_df['salary'].to_numpy()
    lr_unfair.fit(train_df_x, train_df_y)
    
    test_df_x = test_df.drop(columns=['salary'] + protected).to_numpy()
    test_df_y = test_df['salary'].to_numpy()
    preds = lr_unfair.predict(test_df_x)

    # Return Results:
    return preds

In [5]:
# Get the Fair All/L3 Model Predictions
def L3_Model_Replication(train_df, test_df, protected=['sex', 'age', 'race']):
    train_df_protected = train_df[protected].to_numpy()
    test_df_protected = test_df[protected].to_numpy()
    train_df_y = train_df['salary'].to_numpy()
    test_df_y = test_df['salary'].to_numpy()
    
    # workclass
    linear_eps_w = SM_LinearRegression()
    linear_eps_w.fit(train_df_protected, train_df['workclass'])
    eps_w_train = train_df['workclass'].to_numpy() - linear_eps_w.predict(train_df_protected)
    eps_w_test = test_df['workclass'].to_numpy() - linear_eps_w.predict(test_df_protected)
    
    # education-num
    linear_eps_e = SM_LinearRegression()
    linear_eps_e.fit(train_df_protected, train_df['education-num'])
    eps_e_train = train_df['education-num'].to_numpy() - linear_eps_e.predict(train_df_protected)
    eps_e_test = test_df['education-num'].to_numpy() - linear_eps_e.predict(test_df_protected)
    
    # occupation
    linear_eps_o = SM_LinearRegression()
    linear_eps_o.fit(train_df_protected, train_df['occupation'])
    eps_o_train = train_df['occupation'].to_numpy() - linear_eps_o.predict(train_df_protected)
    eps_o_test = test_df['occupation'].to_numpy() - linear_eps_o.predict(test_df_protected)
    
    # capital-gain
    linear_eps_g = SM_LinearRegression()
    linear_eps_g.fit(train_df_protected, train_df['capital-gain'])
    eps_g_train = train_df['capital-gain'].to_numpy() - linear_eps_g.predict(train_df_protected)
    eps_g_test = test_df['capital-gain'].to_numpy() - linear_eps_g.predict(test_df_protected)
    
    # capital-loss
    linear_eps_l = SM_LinearRegression()
    linear_eps_l.fit(train_df_protected, train_df['capital-loss'])
    eps_l_train = train_df['capital-loss'].to_numpy() - linear_eps_l.predict(train_df_protected)
    eps_l_test = test_df['capital-loss'].to_numpy() - linear_eps_l.predict(test_df_protected)
    
    # hours-per-week
    linear_eps_h = SM_LinearRegression()
    linear_eps_h.fit(train_df_protected, train_df['hours-per-week'])
    eps_h_train = train_df['hours-per-week'].to_numpy() - linear_eps_h.predict(train_df_protected)
    eps_h_test = test_df['hours-per-week'].to_numpy() - linear_eps_h.predict(test_df_protected)
    

    # predict on target using abducted latents
    train_eps_stacked = np.hstack((eps_w_train.reshape(-1, 1), eps_e_train.reshape(-1, 1), 
                                   eps_o_train.reshape(-1, 1), eps_g_train.reshape(-1, 1), 
                                   eps_l_train.reshape(-1, 1), eps_h_train.reshape(-1, 1), 
                                   train_df['marital-status'].to_numpy().reshape(-1, 1), 
                                   train_df['relationship'].to_numpy().reshape(-1, 1),
                                   train_df['native-country'].to_numpy().reshape(-1, 1)))
    
    test_eps_stacked = np.hstack((eps_w_test.reshape(-1, 1), eps_e_test.reshape(-1, 1), 
                                  eps_o_test.reshape(-1, 1), eps_g_test.reshape(-1, 1), 
                                  eps_l_test.reshape(-1, 1), eps_h_test.reshape(-1, 1), 
                                  test_df['marital-status'].to_numpy().reshape(-1, 1), 
                                  test_df['relationship'].to_numpy().reshape(-1, 1),
                                  test_df['native-country'].to_numpy().reshape(-1, 1)))
    smlr_L3 = SM_LinearRegression()
    smlr_L3.fit(train_eps_stacked, train_df_y)

    # predict on test epsilons
    preds = smlr_L3.predict(test_eps_stacked)

    # Return Results:
    return preds

In [6]:
unfair_preds = Unfair_Model_Replication(adult_ds.train_df, adult_ds.test_df)

In [7]:
unaware_preds = Unaware_Model_Replication(adult_ds.train_df, adult_ds.test_df)

In [8]:
l3_preds = L3_Model_Replication(adult_ds.train_df, adult_ds.test_df)

In [9]:
print('Unfair RMSE: \t\t\t%.3f' % np.sqrt(mean_squared_error(unfair_preds, adult_ds.test_df['salary'])))
print('FTU RMSE: \t\t\t%.3f' % np.sqrt(mean_squared_error(unaware_preds, adult_ds.test_df['salary'])))
print('Level 3 (Fair Add) RMSE: \t%.3f' % np.sqrt(mean_squared_error(l3_preds, adult_ds.test_df['salary'])))

Unfair RMSE: 			0.348
FTU RMSE: 			0.349
Level 3 (Fair Add) RMSE: 	0.352


## Metrics

### Balanced accuracy

In [10]:
from sklearn.metrics import balanced_accuracy_score

In [11]:
print('Unfair balanced accuracy: \t\t%.3f' % balanced_accuracy_score(adult_ds.test_df['salary'], np.array(unfair_preds >= 0.5).astype(int)))
print('FTU balanced accuracy: \t\t\t%.3f' % balanced_accuracy_score(adult_ds.test_df['salary'], np.array(unaware_preds >= 0.5).astype(int)))
print('Level 3 (Fair Add) balanced accuracy: \t%.3f' % balanced_accuracy_score(adult_ds.test_df['salary'], np.array(l3_preds >= 0.5).astype(int)))

Unfair balanced accuracy: 		0.722
FTU balanced accuracy: 			0.723
Level 3 (Fair Add) balanced accuracy: 	0.716


### Equalized odds

In [12]:
preds_df = adult_ds.test_df.copy()
preds_df['unfair'] = np.array(unfair_preds >= 0.5).astype(int)
preds_df['unaware'] = np.array(unaware_preds >= 0.5).astype(int)
preds_df['l3'] = np.array(l3_preds >= 0.5).astype(int)

In [13]:
def equalized_odds(preds_df, method='unfair', protected=['sex', 'binary_age', 'race']):
    
    preds_df['binary_age'] = preds_df['age'] >= 0.5
    eq_odds = []
    for attribute in protected:
        eq_odd = preds_df.loc[(preds_df['salary']==1) & (preds_df[attribute]==0), method].mean() -  preds_df.loc[(preds_df['salary']==1) & (preds_df[attribute]==1), method].mean()
        eq_odd += preds_df.loc[(preds_df['salary']==0) & (preds_df[attribute]==0), method].mean() -  preds_df.loc[(preds_df['salary']==0) & (preds_df[attribute]==1), method].mean()
        eq_odds.append(np.abs(eq_odd))
    return np.mean(eq_odds)

In [14]:
print('Unfair Equalized Odds score: \t\t\t%.3f' % equalized_odds(preds_df, method='unfair'))
print('FTU Equalized Odds score: \t\t\t%.3f' % equalized_odds(preds_df, method='unaware'))
print('Level 3 (Fair Add) Equalized Odds score: \t%.3f' % equalized_odds(preds_df, method='l3'))

Unfair Equalized Odds score: 			0.220
FTU Equalized Odds score: 			0.147
Level 3 (Fair Add) Equalized Odds score: 	0.083
